In [ ]:
# Import Modules
from legged_gym import LEGGED_GYM_ROOT_DIR
import os
import sys
import isaacgym
from legged_gym.envs import *
from legged_gym.utils import get_args, export_policy_as_jit, task_registry, Logger
from legged_gym.utils.math import quat_apply_yaw, wrap_to_pi, quat_apply
from isaacgym.torch_utils import quat_rotate
import numpy as np
import torch
import pygame

pygame.init()
pygame.joystick.init()
joystick = pygame.joystick.Joystick(0)
joystick.init()

print(f"Joystick name: {joystick.get_name()}")  # 打印手柄名称
print(f"Number of axes: {joystick.get_numaxes()}")  # 打印轴的数量
print(f"Number of buttons: {joystick.get_numbuttons()}")  # 打印按钮的数量

In [ ]:
device = "cuda:0"
sys.argv = sys.argv[0:1]+["--task=wk4_vis"]
# sys.argv = sys.argv[0:1]+["--task=wk4_leg"]
sys.argv.append(f"--sim_device={device}")
sys.argv.append(f"--rl_device={device}")
# sys.argv.append("--headless")
sys.argc = len(sys.argv)
args = get_args()

In [ ]:
env_cfg, train_cfg = task_registry.get_cfgs(name=args.task)
env_cfg.env.num_envs = min(env_cfg.env.num_envs, 4)
# env_cfg.env.num_history_frames = 10

env_cfg.terrain.terrain_proportions = [0.0, 0.0, 0.1, 0.6, 0.2, 0., 0., 0.]
env_cfg.terrain.heightfield_range = [0.0, 0.0]
# env_cfg.terrain.terrain_proportions = [0.]
train_cfg.runner.resume = True
train_cfg.runner.load_run = "240801_184306_Proper_MH_VAE"
train_cfg.runner.checkpoints = -1
env_cfg.terrain.curriculum = False
env_cfg.terrain.mesh_type = 'plane'
env_cfg.terrain.num_rows = 8
env_cfg.terrain.num_cols = 8 
env_cfg.noise.add_noise = False
env_cfg.domain_rand.randomize_friction = False
env_cfg.domain_rand.randomize_base_mass = False
env_cfg.domain_rand.randomize_motor_strength = False
env_cfg.domain_rand.randomize_Kp_factor = False
env_cfg.domain_rand.randomize_Kd_factor = False
env_cfg.domain_rand.push_robots = False
env_cfg.domain_rand.randomize_lag_timesteps = False
env_cfg.domain_rand.randomize_init_state = False

In [ ]:
env, _ = task_registry.make_env(name=args.task, args=args, env_cfg=env_cfg)
for i in range(3):
    _ = env.reset()

In [ ]:
ppo_runner, train_cfg = task_registry.make_alg_runner(env=env, name=args.task, args=args, train_cfg=train_cfg)
policy = ppo_runner.alg.actor_critic

In [ ]:
_ = env.reset()
obs_tuple = env.get_observations()
obs, obs_history, depth_image, est_nominal= obs_tuple
heightmap = est_nominal["heightmap"]

In [ ]:
for i in range(1*int(env.max_episode_length)):
    
    env.cmd_w[:,0] = 0.0
    env.cmd_w[:,1] = 0.0
    env.commands[:,3] = 0
    
    events = pygame.event.get()
    joystick_axes = [joystick.get_axis(i) for i in range(joystick.get_numaxes())]
    joystick_buttons = [joystick.get_button(i) for i in range(joystick.get_numbuttons())]
#     print(f'Axes: {joystick_axes}, Buttons: {joystick_buttons}')
    
    cmd_js = torch.zeros((1,3), device=env.device, requires_grad=False)
    cmd_js[0,0] = -joystick_axes[1] * 0.8
    cmd_js[0,1] = -joystick_axes[0] * 0.2
    env.cmd_w[0,:] = quat_rotate(env.base_quat[0:1], cmd_js)
    forward = quat_apply(env.base_quat, env.forward_vec)
    heading = torch.atan2(forward[:, 1], forward[:, 0])
    env.commands[0,3] = heading[0] - joystick_axes[2]*2.5
    
    
#     actions = policy(obs.detach(), obs_history.detach(), depth_image.detach())
    actions = policy(obs.detach(), obs_history.detach(), heightmap=heightmap)
    obs_tuple, rewards, dones, infos = env.step(actions)
    obs, privileged_obs, obs_history, depth_image, est_nominal, obs_future = obs_tuple
#     obs, privileged_obs, obs_history, est_nominal, obs_future = obs_tuple
    
    heightmap = est_nominal["heightmap"]
#     depth_image = torch.zeros((env.num_envs, 2, 60, 106), device=env.device)
#     print(obs[:,6:9], env.commands, "\n")
#     print(joystick_axes[2],env.commands[0,3].item(),heading[0].item(), obs[0,8].item())
ppo_runner.alg.actor_critic.reset(dones)

In [20]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib auto
%matplotlib auto

Using matplotlib backend: <object object at 0x7fbb38922c20>
Using matplotlib backend: TkAgg


In [ ]:
for robot_to_check in range(12,16):
    fig, axs = plt.subplots(1,2, figsize=(14,6))
    img = axs[0].imshow(depth_image[robot_to_check,0].cpu(), cmap="gray", vmin=-0.5, vmax=0.5)
    axs[1].imshow(depth_image[robot_to_check,1].cpu(), cmap="gray", vmin=-0.5, vmax=0.5)
    fig.colorbar(img)
    axs[1].set_title(robot_to_check)
    fig.tight_layout()
    plt.show()

In [22]:
idx = 2
return_tuple = policy.ce_net(obs_history.detach(), heightmap.detach())

In [24]:
fig = plt.figure()
xx, yy = np.meshgrid(np.arange(18), np.arange(11))

for idx in range(4):
    hm_nom = est_nominal['heightmap'][idx].cpu().detach().numpy().reshape(18,11).T
    hm_est = return_tuple[1][1][idx].cpu().detach().numpy().reshape(18,11).T
    ax_max = fig.add_subplot(221+idx, projection='3d')

    ax_max.scatter(xx, yy, hm_nom, label = "nom")
    ax_max.scatter(xx, yy, hm_est, label = "est")
    ax_max.set_title(f"heightmap_{idx}")
    zlim = ax_max.get_zlim()
    if zlim[1]-zlim[0] < 0.5:
        print(idx, zlim)
        ax_max.set_zlim([zlim[0]-0.5, zlim[1]+0.5])
ax_max.legend()
fig.tight_layout()
plt.show()

0 (-0.5491507470607757, -0.5154244482517243)
1 (-0.7737729758024215, -0.7346065431833267)
2 (-0.8024385541677475, -0.7626209288835526)
3 (-0.38667471408843995, -0.357023286819458)


In [ ]:
for i in range(10*int(env.max_episode_length)):
    actions = policy(obs.detach(), obs_history.detach(), depth_image.detach())
    obs_tuple, rewards, dones, infos = env.step(actions)
    env.cmd_w[:,:3] = 0
    obs, privileged_obs, obs_history, depth_image, est_nominal, obs_future = obs_tuple

In [ ]:
x = np.arange(0,10000)
y = 0.998**x
z = np.exp(-x*1e-3)
plt.plot(x,y,label="idx")
plt.plot(x,z,label="exp")
plt.grid("on")
plt.legend()
plt.show()

print(np.sum(y>1e-2))
print(np.sum(y>1e-3))
print(np.sum(y>1e-4))

In [ ]:

plt.plot(x,y-z,label="exp")
plt.show()